In [1]:
from datetime import datetime
import fsspec
from http.cookiejar import CookieJar
from netrc import netrc
import numpy as np
from os.path import basename, isfile, isdir, join, expanduser
from platform import system
from urllib import request
import xarray as xr
import time
import requests
import matplotlib.pyplot as plt
import cartopy
import dask
import requests,s3fs
import numpy.ma as ma
import matplotlib
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pandas as pd
from pathlib import Path 
import glob, os

# Subroutines

In [2]:
def z_masked_overlap(axe, X, Y, Z, source_projection=None):
    """
    for data in projection axe.projection
    find and mask the overlaps (more 1/2 the axe.projection range)

    X, Y either the coordinates in axe.projection or longitudes latitudes
    Z the data
    operation one of 'pcorlor', 'pcolormesh', 'countour', 'countourf'

    if source_projection is a geodetic CRS data is in geodetic coordinates
    and should first be projected in axe.projection

    X, Y are 2D same dimension as Z for contour and contourf
    same dimension as Z or with an extra row and column for pcolor
    and pcolormesh

    return ptx, pty, Z
    """
    if not hasattr(axe, 'projection'):
        return Z
    if not isinstance(axe.projection, cartopy.crs.Projection):
        return Z

    if len(X.shape) != 2 or len(Y.shape) != 2:
        return Z

    if (source_projection is not None and
            isinstance(source_projection, cartopy.crs.Geodetic)):
        transformed_pts = axe.projection.transform_points(
            source_projection, X, Y)
        ptx, pty = transformed_pts[..., 0], transformed_pts[..., 1]
    else:
        ptx, pty = X, Y


    with np.errstate(invalid='ignore'):
        # diagonals have one less row and one less columns
        diagonal0_lengths = np.hypot(
            ptx[1:, 1:] - ptx[:-1, :-1],
            pty[1:, 1:] - pty[:-1, :-1]
        )
        diagonal1_lengths = np.hypot(
            ptx[1:, :-1] - ptx[:-1, 1:],
            pty[1:, :-1] - pty[:-1, 1:]
        )
        to_mask = (
            (diagonal0_lengths > (
                abs(axe.projection.x_limits[1]
                    - axe.projection.x_limits[0])) / 2) |
            np.isnan(diagonal0_lengths) |
            (diagonal1_lengths > (
                abs(axe.projection.x_limits[1]
                    - axe.projection.x_limits[0])) / 2) |
            np.isnan(diagonal1_lengths)
        )

        # TODO check if we need to do something about surrounding vertices

        # add one extra colum and row for contour and contourf
        if (to_mask.shape[0] == Z.shape[0] - 1 and
                to_mask.shape[1] == Z.shape[1] - 1):
            to_mask_extended = np.zeros(Z.shape, dtype=bool)
            to_mask_extended[:-1, :-1] = to_mask
            to_mask_extended[-1, :] = to_mask_extended[-2, :]
            to_mask_extended[:, -1] = to_mask_extended[:, -2]
            to_mask = to_mask_extended
        if np.any(to_mask):

            Z_mask = getattr(Z, 'mask', None)
            to_mask = to_mask if Z_mask is None else to_mask | Z_mask

            Z = ma.masked_where(to_mask, Z)

        return ptx, pty, Z
        

In [3]:
def pcolormesh_part(LO,LA,data,valmin,valmax,lon0,lonmin,lonmax,latmin,latmax,**karg):

    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=lon0))
    
    ax.set_extent([lonmin, lonmax, latmin, latmax], crs=cartopy.crs.PlateCarree()) 
    ax.add_feature(cfeature.LAND, facecolor = '0.75',zorder=1)
    ax.coastlines('10m',zorder=2)
    ax.add_feature(cfeature.RIVERS,facecolor='blue',zorder=3)

    gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.xlocator = mticker.FixedLocator(np.arange(lonmin,lonmax,15))
    gl.xlabel_style = {'size': 20, 'color': 'k','rotation':0}

    gl.yformatter = LATITUDE_FORMATTER
    gl.ylocator = mticker.FixedLocator(np.arange(latmin,latmax,5))
    gl.ylabel_style = {'size': 20, 'color': 'gray','rotation':0}
    
    palette = plt.cm.jet
    palette2 = plt.cm.Greys

    pp=ax.pcolormesh(LO.squeeze(), LA.squeeze(), data, cmap=palette, transform=cartopy.crs.PlateCarree(), vmin = valmin, vmax = valmax)

    if 'data2' in karg:
        pp2=ax.pcolormesh(karg['LO2'].squeeze(), karg['LA2'].squeeze(), karg['data2'], cmap=palette2, transform=cartopy.crs.PlateCarree(), vmin = karg['valmin2'], vmax = karg['valmax2'])
        
    if 'contour' in karg:  
        matplotlib.rcParams['contour.negative_linestyle'] = 'solid'
        X, Y, masked_data = z_masked_overlap(ax, karg['lon_contour'], karg['lat_contour'], karg['contour'], source_projection=cartopy.crs.Geodetic())    
        cc=ax.contour(X,Y,masked_data,levels=[karg['level_contour']],colors=karg['color_contour'],linewidth=3)
        if 'str_contour' in karg:  
            fmt = {}
            for l, s in zip(cc.levels, karg['str_contour']):
                fmt[l] = s
            plt.clabel(cc, cc.levels, fmt=fmt, inline=True, fontsize=10,colors=karg['color_contour'])
        else:
            plt.clabel(cc, cc.levels, fmt=' {:.0f} '.format, inline=True, fontsize=10,colors=karg['color_contour'])
    
    if 'uvector' in karg:  
        q=plt.quiver(karg['lonvector'],karg['latvector'],karg['uvector'],karg['vvector'], scale=karg['scale'], transform=cartopy.crs.PlateCarree())
        qk= plt.quiverkey (q,0.95, 1.02,10, karg['strvector'], labelpos='N')
        
    cbar_ax = fig.add_axes([0.82, 0.18, 0.03, 0.78])
    h=plt.colorbar(pp, cax=cbar_ax,orientation='vertical',ax=ax)
    h.ax.tick_params(labelsize=16)
    cbar_ax = fig.add_axes([0.9, 0.18, 0.03, 0.78])
    h2=plt.colorbar(pp2, cax=cbar_ax,orientation='vertical',ax=ax)
    h2.ax.tick_params(labelsize=16)
    if 'unit' in karg:
        if 'fontsize_unit' in karg:
            h.set_label(karg['unit'],fontsize=karg['fontsize_unit'])
        else:
            h.set_label(karg['unit'],fontsize=20)
    if 'unit2' in karg:
        if 'fontsize_unit2' in karg:
            h2.set_label(karg['unit2'],fontsize=karg['fontsize_unit2'])
        else:
            h2.set_label(karg['unit2'],fontsize=20)

    if 'title' in karg:
        plt.suptitle(karg['title'],fontsize=20)
        plt.subplots_adjust(right=0.9,left=0.1,top=0.9,bottom=0.15)
        gl.top_labels =False
        gl.right_labels =False
    else:
        plt.subplots_adjust(right=0.9,left=0.1,top=0.95,bottom=0.15)

    plt.subplots_adjust(right=0.9,left=0.1,top=0.95,bottom=0.18)
    if 'fileout' in karg:
        plt.ioff()
        plt.savefig(karg['fileout'], dpi=fig.dpi)
        plt.close('all')
    else:
        plt.ion()
        plt.show()

In [4]:
def init_S3FileSystem():
    """
    This routine automatically pulls your EDL crediential from .netrc file and use it to obtain an AWS S3 credential 
    through a PO.DAAC service accessible at https://archive.podaac.earthdata.nasa.gov/s3credentials.
    From the PO.DAAC Github (https://podaac.github.io/tutorials/external/July_2022_Earthdata_Webinar.html).
    
    Returns:
    =======
    
    s3: an AWS S3 filesystem
    """
  
    import requests,s3fs
    credentials = requests.get('https://archive.podaac.earthdata.nasa.gov/s3credentials').json()
    s3 = s3fs.S3FileSystem(anon=False,
                           key=credentials['accessKeyId'],
                           secret=credentials['secretAccessKey'], 
                           token=credentials['sessionToken'])
    return s3, credentials


In [5]:
def update_credential(credentials, force=False):
    now=np.datetime64(datetime.now())
    
    # expiration time of current credential
    exp=np.datetime64(credentials['expiration'][:-6])
    # current time
    
    # how much time is left [seconds]
    td_sec = np.double(exp-now)/1e6

    # if < 1800 seconds left before credential expires, renew it
    if (td_sec < 1800) or (force==True):
        print(f'... updating credentials, {td_sec}s remaining')
        s3, credentials = init_S3FileSystem()
        exp=np.datetime64(credentials['expiration'][:-6])
        td_sec = np.double(exp-now)/1e6
        print(f'... after credential update, {td_sec}s remaining')
    else:
        print(f'... not updating credentials, {td_sec}s remaining')
        
    return credentials

In [6]:
# do we need that?
# _netrc = join(expanduser('~'), "_netrc" if system()=="Windows" else ".netrc")
# print(_netrc, type(_netrc))

# setup_earthdata_login_auth()

In [7]:
s3, credentials = init_S3FileSystem()

In [2]:
# update_credential(credentials, force=False)

# Load SMAP files

In [9]:
ShortName = "SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6"
year=2023
month=8
doystart=datetime(year,month,1).timetuple().tm_yday
doyend=datetime(year,11,30).timetuple().tm_yday
smap_files = s3.glob(join("podaac-ops-cumulus-protected/", ShortName, '*'+str(year)+'_*.nc'))
paths=[s3.open(f) for f in smap_files]
paths[doystart-1:doystart+10]

[<File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6/RSS_smap_SSS_L3_8day_running_2023_213_FNL_v06.0.nc>,
 <File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6/RSS_smap_SSS_L3_8day_running_2023_214_FNL_v06.0.nc>,
 <File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6/RSS_smap_SSS_L3_8day_running_2023_215_FNL_v06.0.nc>,
 <File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6/RSS_smap_SSS_L3_8day_running_2023_216_FNL_v06.0.nc>,
 <File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6/RSS_smap_SSS_L3_8day_running_2023_217_FNL_v06.0.nc>,
 <File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V6/RSS_smap_SSS_L3_8day_running_2023_218_FNL_v06.0.nc>,
 <File-like object S3FileSystem, podaac-ops-cumulus-protected/SMAP_RSS

In [10]:
start_time = time.time()
smap = xr.open_mfdataset(paths[doystart-1:doyend+1],
    combine='nested',
    concat_dim='time',
    decode_cf=True,
    coords='minimal',
    chunks={'time': 1}) 

print(time.time() - start_time)
print((time.time() - start_time)/len(paths))

40.900121212005615
0.11205553551242776


In [11]:
smap

<xarray.Dataset> Size: 19GB
Dimensions:                 (lon: 1440, lat: 720, time: 123,
                             uncertainty_components: 9, iceflag_components: 3)
Coordinates:
  * lon                     (lon) float32 6kB 0.125 0.375 0.625 ... 359.6 359.9
  * lat                     (lat) float32 3kB -89.88 -89.62 ... 89.62 89.88
  * time                    (time) datetime64[ns] 984B 2023-08-01T12:00:00 .....
Dimensions without coordinates: uncertainty_components, iceflag_components
Data variables: (12/19)
    nobs                    (time, lat, lon) float64 1GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    nobs_RF                 (time, lat, lon) float64 1GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    nobs_40km               (time, lat, lon) float64 1GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sss_smap                (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sss_smap_RF             (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sss_smap_unc            (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ...                      ...
    fland                   (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    gice_est                (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    surtep                  (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    winspd                  (time, lat, lon) float32 510MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_zones           (time, lat, lon) int8 128MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    anc_sea_ice_flag        (time, lat, lon, iceflag_components) int8 383MB dask.array<chunksize=(1, 720, 1440, 2), meta=np.ndarray>
Attributes: (12/65)
    Conventions:                                            CF-1.7, ACDD-1.3
    title:                                                  SMAP ocean surfac...
    version:                                                V6.0 Validated Re...
    summary:                                                The dataset conta...
    acknowledgement:                                        Funded under Subc...
    processing_level:                                       L3
    ...                                                     ...
    Source_of_SMAP_SSS_retrievals:                          T. Meissner, F. W...
    Source_of_ancillary_SST:                                Canada Meteorolog...
    Source_of_ancillary_CCMP_wind_speed:                    Mears, C. et al.,...
    Source_of_ancillary_AMSR2_sea_ice_flag_and_correction:  Meissner, T. and ...
    Source_of_ancillary_land_mask:                          1 km land/water m...
    Source_of_ancillary_reference_SSS_from_HYCOM:           Hybrid Coordinate...

# Load SIC files

In [12]:
sic_path=Path('/home/jpluser/efs-mount-point/sevfour/sic/')
files = sorted(glob.glob(os.path.join(sic_path, '*'+str(year)+'*.nc')))
files[0:10]

['/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230801_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230802_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230803_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230804_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230805_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230806_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230807_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230808_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230809_F17_v05r00.nc',
 '/home/jpluser/efs-mount-point/sevfour/sic/sic_psn25_20230810_F17_v05r00.nc']

In [13]:
grid_sic=xr.open_dataset(Path('/home/jpluser/efs-mount-point/sevfour/grid_sic_nsidc.nc'))

/home/jpluser/miniforge3/envs/jupyter/lib/python3.11/site-packages/xarray/conventions.py:440: SerializationWarning: variable 'melt_onset_day_seaice_conc_cdr' has multiple fill values {-1, 65535}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [14]:
sic = xr.open_mfdataset(
        paths=files,
        combine='nested',
        concat_dim='time') 

In [15]:
sic.cdr_seaice_conc

<xarray.DataArray 'cdr_seaice_conc' (time: 122, y: 448, x: 304)> Size: 66MB
dask.array<concatenate, shape=(122, 448, 304), dtype=float32, chunksize=(1, 448, 304), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 2kB -3.838e+06 -3.812e+06 ... 3.712e+06 3.738e+06
  * y        (y) float64 4kB 5.838e+06 5.812e+06 ... -5.312e+06 -5.338e+06
  * time     (time) datetime64[ns] 976B 2023-08-01 2023-08-02 ... 2023-11-30
Attributes:
    standard_name:          sea_ice_area_fraction
    coverage_content_type:  image
    units:                  1
    long_name:              NOAA/NSIDC CDR of Passive Microwave Sea Ice Conce...
    grid_mapping:           crs
    ancillary_variables:    cdr_seaice_conc_stdev cdr_seaice_conc_qa_flag
    valid_range:            [  0 100]

In [16]:
sic=sic.where(sic.cdr_seaice_conc > 0.15)

# Figures

In [17]:
lon0=-140
lonmapmin=-157
lonmapmax=-125
latmapmin=67
latmapmax=80
land=True

In [20]:
rr=0
for i in np.arange(0,smap.sss_smap.shape[1]+1,4):
    pcolormesh_part(smap.lon,smap.lat,smap.sss_smap[i,:,:],20,35,lon0,lonmapmin,lonmapmax,latmapmin,latmapmax
                ,data2=sic.cdr_seaice_conc[i,:,:]*100,LO2=grid_sic.longitude,LA2=grid_sic.latitude,valmin2=0,valmax2=100
                ,title=str(pd.to_datetime(smap.time[i].values).date())
                ,fileout='/home/jpluser/Figures/FRESH/smap_sic_'+str(year)+'_'+str(rr).zfill(3)+'.png'
           )
    rr=rr+1

/home/jpluser/miniforge3/envs/jupyter/lib/python3.11/site-packages/cartopy/mpl/style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
/home/jpluser/miniforge3/envs/jupyter/lib/python3.11/site-packages/cartopy/mpl/style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
/home/jpluser/miniforge3/envs/jupyter/lib/python3.11/site-packages/cartopy/mpl/style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
/home/jpluser/miniforge3/envs/jupyter/lib/python3.11/site-packages/cartopy/mpl/style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
/home/jpluser/miniforge3/envs/jupyter/lib/python3.11/site-packag

IndexError: index 124 is out of bounds for axis 0 with size 123